In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import shutil

CLASS_MAP = {
    'Caries': 0,
    'Cavity': 1,
    'Crack': 2,
    'Tooth':3
}

DRIVE_PATH = '/content/drive/MyDrive/'
DATASET_ROOT_PATH = os.path.join(DRIVE_PATH, 'TALROP/Dental_dataset')
OUTPUT_DIR = os.path.join(DRIVE_PATH, 'Dental_YOLO_Final_Dataset')

Creating Dirs. in O/P folder

In [3]:
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(OUTPUT_DIR, f'images/{split}'), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, f'labels/{split}'), exist_ok=True)

Processing Logic [Make list of images -> loop through those images -> Match image to ann -> Extract img dim -> Runs a loop through the objects(in a json file) -> Translation (class name to number)]

In [4]:
def process_split(split_name):
  src_img_dir = os.path.join(DATASET_ROOT_PATH, split_name, 'img/')
  src_ann_dir = os.path.join(DATASET_ROOT_PATH, split_name, 'ann/')

  filenames = [f for f in os.listdir(src_img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
  total_files = len(filenames)

  for i, img_filename in enumerate(filenames):
    base_filename = os.path.splitext(img_filename)[0]
    json_filename = f"{img_filename}.json"

    src_img_path = os.path.join(src_img_dir, img_filename)
    src_ann_path = os.path.join(src_ann_dir, json_filename)

    if not os.path.exists(src_ann_path):
      print(f"\nWarning: Annotation for '{img_filename}' not found. Skipping this file.")
      continue


    with open(src_ann_path, 'r') as f:
      ann_data = json.load(f)

    img_width = ann_data['size']['width']
    img_height = ann_data['size']['height']

    yolo_labels = []

    for obj in ann_data['objects']:
      class_title = obj['classTitle']
      if class_title in CLASS_MAP:
        class_index = CLASS_MAP[class_title]
        points = obj['points']['exterior']

        normalized_points = []
        for x,y in points:
          norm_x = x / img_width
          norm_y = y / img_height
          normalized_points.extend([norm_x, norm_y])

        if normalized_points:
          yolo_line = f"{class_index} " + " ".join(f"{p:.6f}" for p in normalized_points)
          yolo_labels.append(yolo_line)


    if yolo_labels:
      dest_img_path = os.path.join(OUTPUT_DIR, f'images/{split_name}/{img_filename}')
      dest_label_path = os.path.join(OUTPUT_DIR, f'labels/{split_name}/{base_filename}.txt')
      with open(dest_label_path, 'w') as f:
        f.write("\n".join(yolo_labels))
      shutil.copy2(src_img_path, dest_img_path)

In [5]:
process_split('train')
process_split('valid')
process_split('test')

print("\n----------------------------------")
print("ALL PREPROCESSING IS COMPLETE!")
print(f"Your final, model-ready dataset is now in: {OUTPUT_DIR}")
print("----------------------------------")












----------------------------------
✅ ALL PREPROCESSING IS COMPLETE!
Your final, model-ready dataset is now in: /content/drive/MyDrive/Dental_YOLO_Final_Dataset
----------------------------------


In [6]:
yaml_content = """

train: /content/drive/MyDrive/Dental_YOLO_Final_Dataset/images/train
val: /content/drive/MyDrive/Dental_YOLO_Final_Dataset/images/valid
test: /content/drive/MyDrive/Dental_YOLO_Final_Dataset/images/test

nc: 4
names: ['Caries', 'Cavity', 'Crack', 'Tooth']
"""

with open('dental_dataset.yaml', 'w') as f:
    f.write(yaml_content)

In [7]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [8]:
from ultralytics import YOLO

# Load a pretrained model to start training
model = YOLO('yolov8n-seg.pt')

# Start the training and stop automatically at 20 epochs
results = model.train(
    data='dental_dataset.yaml',
    epochs=100,      # <-- Set to the target for Day 1
    imgsz=640,
    project='Dental_YOLO_Training',
    name='automated_run_100_epochs' # A consistent name for the project
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dental_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=automated_run_100_epochs, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n-seg summary: 151 layers, 3,264,396 parameters, 3,264,380 gradients, 12.1 GFLOPs

Transferred 381/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.4±0.1 ms, read: 22.3±24.9 MB/s, size: 260.2 KB)


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:02<00:07,  1.25s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:13<00:00,  1.74s/it]


                   all        254       3156      0.576      0.444      0.457       0.31      0.546      0.425      0.429      0.278
                Caries        102        328      0.546      0.207      0.254       0.11      0.514      0.195      0.252      0.104
                Cavity         85        159      0.475      0.346      0.377      0.192      0.397      0.289       0.28      0.125
                 Crack         12         16      0.439       0.25      0.223     0.0965      0.439       0.25       0.22        0.1
                 Tooth        254       2653      0.843      0.973      0.975      0.843      0.835      0.964      0.964      0.782
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 7.7ms postprocess per image
Results saved to Dental_YOLO_Training/automated_run_100_epochs


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dental_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Dental_YOLO_Training/automated_run_100_epochs/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=automated_run_100_epochs3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

train: Scanning /content/drive/MyDrive/Dental_YOLO_Final_Dataset/labels/train.cache... 1991 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1991/1991 [00:00<?, ?it/s]

train: /content/drive/MyDrive/Dental_YOLO_Final_Dataset/images/train/804_jpg.rf.d6fb1371223a94b6caaaab80cd7aaac7.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.8±0.4 ms, read: 87.4±94.2 MB/s, size: 379.2 KB)


val: Scanning /content/drive/MyDrive/Dental_YOLO_Final_Dataset/labels/valid.cache... 254 images, 0 backgrounds, 0 corrupt: 100%|██████████| 254/254 [00:00<?, ?it/s]


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:02<00:07,  1.19s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:14<00:00,  1.77s/it]


                   all        254       3156       0.64      0.438      0.489      0.332      0.624      0.423      0.458      0.293
                Caries        102        328      0.614      0.189      0.279      0.121      0.599       0.18      0.268       0.11
                Cavity         85        159      0.594      0.352      0.403      0.205      0.545      0.308      0.301      0.135
                 Crack         12         16      0.471       0.25      0.302       0.16      0.477       0.25      0.302      0.143
                 Tooth        254       2653      0.879      0.963      0.972      0.843      0.873      0.954      0.961      0.783
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 7.3ms postprocess per image
Results saved to Dental_YOLO_Training/automated_run_100_epochs3


NameError: name 'YOLO' is not defined